#Connect to sharepoint

In [1]:
!pip install office365
!pip install Office365-REST-Python-Client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.files.file import File

####inputs########
# This will be the URL that points to your sharepoint site.
# Make sure you change only the parts of the link that start with "Your"
url_shrpt = 'https://ihuedu.sharepoint.com/sites/EDYTEProject2023/'
username_shrpt = '###############'
password_shrpt = '###############'
folder_files_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/entities_extraction/'
folder_analysis_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/'

In [3]:
###Authentication###For authenticating into your sharepoint site###
ctx_auth = AuthenticationContext(url_shrpt)
if ctx_auth.acquire_token_for_user(username_shrpt, password_shrpt):
  ctx = ClientContext(url_shrpt, ctx_auth)
  web = ctx.web
  ctx.load(web)
  ctx.execute_query()
  print('Authenticated into sharepoint as: ',web.properties['Title'])

else:
  print(ctx_auth.get_last_error())
############################

Authenticated into sharepoint as:  EDYTE Project 2023


In [4]:
####Function for extracting the file names of a folder in sharepoint###
###If you want to extract the folder names instead of file names, you have to change "sub_folders = folder.files" to "sub_folders = folder.folders" in the below function

global print_folder_contents
def print_folder_contents(ctx, folder_url):
    try:

        folder = ctx.web.get_folder_by_server_relative_url(folder_url)
        fold_names = []
        sub_folders = folder.files #Replace files with folders for getting list of folders
        ctx.load(sub_folders)
        ctx.execute_query()

        for s_folder in sub_folders:

            fold_names.append(s_folder.properties["Name"])

        return fold_names

    except Exception as e:
        print('Problem printing out library contents: ', e)
######################################################

# Call the function by giving your folder URL as input
filelist_shrpt=print_folder_contents(ctx,folder_files_url_shrpt)
#Print the list of files present in the folder
print(filelist_shrpt)

['entities_topic.csv', 'Entities_Clustering.ipynb', 'conditions_entities.csv', 'Entities_Extraction.ipynb', 'steps_entities.csv', 'Entities_Disambiguation.ipynb', 'final_entities_dic.csv', 'steps_digital_entities.csv', 'process_entities.csv', 'evidences_entities.csv']


# Data

In [5]:
import pandas as pd
import csv
import numpy as np
import io
import os
import tempfile

In [6]:
def process_csv_file(file_name):

    file_url= folder_files_url_shrpt+file_name

    response = File.open_binary(ctx, file_url)  # Assuming File is imported and ctx is defined

    df = pd.read_csv(io.BytesIO(response.content))

    return df

In [7]:
combined = process_csv_file ("entities_topic.csv")

In [8]:
combined

,index,id,ihu_unique_step_id,ihu_unique_step_digital_id,ihu_unique_evidence_id,ihu_unique_condition_id,word,topic
0,0,523764.0,NaN,NaN,NaN,NaN,ελγα,133
1,1,523764.0,NaN,NaN,NaN,NaN,οργανισμου ελληνικων γεωργικων ασφαλισεων,44
2,3,963466.0,NaN,NaN,NaN,NaN,ιδιωτικων επιχειρησεων παροχης υπηρεσιων ασφαλ...,449
3,4,963466.0,NaN,NaN,NaN,NaN,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ...",-1
4,5,439605.0,NaN,NaN,NaN,NaN,οργανισμου ελληνικων γεωργικων ασφαλισεων,44
...,...,...,...,...,...,...,...,...
55271,63224,NaN,NaN,NaN,NaN,135964_con_01,φορτηγο δημοσιας,80
55272,63225,NaN,NaN,NaN,NaN,135964_con_06,ιδιωτικης,285
55273,63226,NaN,NaN,NaN,NaN,373136_con_01,φορτηγο,60
55274,63227,NaN,NaN,NaN,NaN,373136_con_01,αυτοκινητου,60


#Import Models

In [9]:
!pip install faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=554bd01b58e494857a43e630e57622ce4e3340162b4ab1057b61a602d2888657
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [10]:
import pandas as pd

In [11]:
import torch
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss

# Analysis on Entities

In [12]:
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

def get_sentence_embeddings(sentences, model):
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True, batch_size=8)
    return sentence_embeddings.cpu()

def build_faiss_index(embeddings):
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    return index

def find_most_similar_sentence(query_embedding, index, embeddings, similarity_threshold, kept_sentences):
    similarity_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    top_similarities = similarity_scores[similarity_scores >= similarity_threshold]
    if len(top_similarities) > 0:
        max_similarity = top_similarities.max().item()
        max_similarity_index = np.argmax(similarity_scores)
        if max_similarity >= similarity_threshold:
            return kept_sentences[max_similarity_index], max_similarity
    return '', 0.0

def filter_similar_sentences(dataframe, model, similarity_threshold=0.95):
    # Create new columns to store the most similar sentence and similarity measure
    dataframe['Final_Entity'] = ''
    dataframe['Similarity_Measure'] = 0.0

    # Initialize the kept_sentences list and embeddings
    kept_sentences = []
    kept_embeddings = np.empty((0, model.get_sentence_embedding_dimension()), dtype=np.float32)
    kept_index = None

    # Iterate over the rows in the dataframe with progress bar
    for i, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Processing"):
        curr_sentence = row['word']
        if isinstance(curr_sentence, str):  # Check if the sentence is a valid string
            curr_embedding = get_sentence_embeddings([curr_sentence], model)[0]  # Pass the model as an argument
            most_similar_sentence = ''
            max_similarity = 0.0

            # Compare with the previously kept sentences using FAISS index
            if kept_index is not None:
                most_similar_sentence, max_similarity = find_most_similar_sentence(
                    curr_embedding, kept_index, kept_embeddings, similarity_threshold, kept_sentences
                )

            # If a similar sentence is found, update the most similar sentence and similarity measure columns
            if most_similar_sentence != '':
                dataframe.loc[i, 'Final_Entity'] = most_similar_sentence
                dataframe.loc[i, 'Similarity_Measure'] = max_similarity
            else:
                # If no similar sentence is found, keep the original text instead of the embedding
                kept_sentences.append(curr_sentence)
                dataframe.loc[i, 'Final_Entity'] = curr_sentence
                dataframe.loc[i, 'Similarity_Measure'] = 1.0  # Set similarity measure to 1.0 for the original sentence

            # Check if the current sentence is similar to the kept sentences
            if most_similar_sentence == '' or max_similarity < similarity_threshold:
                kept_embeddings = np.vstack((kept_embeddings, curr_embedding.cpu().numpy()))
                if kept_index is None:
                    kept_index = build_faiss_index(np.array([curr_embedding.cpu().numpy()]))
                else:
                    kept_embeddings_array = np.array(kept_embeddings)
                    new_embeddings_array = np.array([curr_embedding.cpu().numpy()])
                    combined_embeddings = np.vstack((kept_embeddings_array, new_embeddings_array))
                    kept_index.add(combined_embeddings)

        else:
            dataframe.loc[i, 'Final_Entity'] = np.nan
            dataframe.loc[i, 'Similarity_Measure'] = np.nan

    return dataframe


In [13]:
topics_num = combined["topic"].nunique()-1
print(topics_num)

606


In [14]:
model_name = 'paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name)
#homogenised_entities = filter_similar_sentences(combined, model, similarity_threshold=0.975)

In [15]:
homogenised_entities = pd.DataFrame()

for j in range(-1, topics_num):

  sentences = combined.loc[combined['topic'] == j]
  filtered_df = filter_similar_sentences(sentences, model, similarity_threshold=0.975)

  homogenised_entities = pd.concat([homogenised_entities, filtered_df], ignore_index=True)

Streaming output truncated to the last 5000 lines.
<ipython-input-12-9785463df406>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Similarity_Measure'] = 0.0
Processing: 100%|██████████| 78/78 [00:01<00:00, 59.49it/s]
<ipython-input-12-9785463df406>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Final_Entity'] = ''
<ipython-input-12-9785463df406>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [16]:
homogenised_entities

,index,id,ihu_unique_step_id,ihu_unique_step_digital_id,ihu_unique_evidence_id,ihu_unique_condition_id,word,topic,Final_Entity,Similarity_Measure
0,4,963466.0,NaN,NaN,NaN,NaN,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ...",-1,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ...",1.000000
1,18,268173.0,NaN,NaN,NaN,NaN,και επισκευαστες κλειδαριων,-1,και επισκευαστες κλειδαριων,1.000000
2,63,871426.0,NaN,NaN,NaN,NaN,εθνικο μητρωο εθελοντων αιμοδοτων,-1,εθνικο μητρωο εθελοντων αιμοδοτων,1.000000
3,72,233198.0,NaN,NaN,NaN,NaN,ειδικους εκλογικους καταλογους,-1,ειδικους εκλογικους καταλογους,1.000000
4,80,251271.0,NaN,NaN,NaN,NaN,μητρωο αρνητων δωρεας,-1,μητρωο αρνητων δωρεας,1.000000
...,...,...,...,...,...,...,...,...,...,...
55271,60867,NaN,NaN,NaN,NaN,948275_con_01,οργανωσης,605,οργανωσης,1.000000
55272,60943,NaN,NaN,NaN,NaN,426368_con_02,διοικητικο,605,διοικητικα,0.997163
55273,61038,NaN,NaN,NaN,NaN,936695_con_02,διοικητικο,605,διοικητικα,0.997163
55274,61046,NaN,NaN,NaN,NaN,228930_con_03,διοικητικου,605,διοικητικου,1.000000


# Assign ID & Save

In [17]:
def add_entity_ids(df):
    # Extract unique keywords
    keywords = set(df['Final_Entity'])

    # Assign IDs to keywords
    keyword_ids = {keyword: i for i, keyword in enumerate(keywords, start=1)}

    # Create the dictionary of unique keywords and IDs
    keyword_dict = {keyword: keyword_ids[keyword] for keyword in keywords}

    # Add the "Entity ID" column to the dataframe
    df['Entity ID'] = df['Final_Entity'].apply(lambda x: keyword_ids[x])

    return df, keyword_dict

In [18]:
def upload_ids_to_target_folder(dic, name):

  path = name+".csv"

  # open file for writing, "w" is writing
  w = csv.writer(open(path, "w"))

  # loop over dictionary keys and values
  for key, val in dic.items():

      # write every key and value to file
      w.writerow([key, val])

  url=folder_analysis_url_shrpt+"entities_extraction"
  target_folder = ctx.web.get_folder_by_server_relative_url(url)

  with open(path, "rb") as content_file:
      file_content = content_file.read()
      target_folder.upload_file(os.path.basename(path), file_content).execute_query()

In [35]:
final_entities, entities_dict = add_entity_ids(homogenised_entities)
final_entities = final_entities.drop_duplicates()

In [36]:
final_entities

,index,id,ihu_unique_step_id,ihu_unique_step_digital_id,ihu_unique_evidence_id,ihu_unique_condition_id,word,topic,Final_Entity,Similarity_Measure,Entity ID
0,4,963466.0,NaN,NaN,NaN,NaN,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ...",-1,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ...",1.000000,3839
1,18,268173.0,NaN,NaN,NaN,NaN,και επισκευαστες κλειδαριων,-1,και επισκευαστες κλειδαριων,1.000000,10036
2,63,871426.0,NaN,NaN,NaN,NaN,εθνικο μητρωο εθελοντων αιμοδοτων,-1,εθνικο μητρωο εθελοντων αιμοδοτων,1.000000,8239
3,72,233198.0,NaN,NaN,NaN,NaN,ειδικους εκλογικους καταλογους,-1,ειδικους εκλογικους καταλογους,1.000000,7880
4,80,251271.0,NaN,NaN,NaN,NaN,μητρωο αρνητων δωρεας,-1,μητρωο αρνητων δωρεας,1.000000,5342
...,...,...,...,...,...,...,...,...,...,...,...
55271,60867,NaN,NaN,NaN,NaN,948275_con_01,οργανωσης,605,οργανωσης,1.000000,569
55272,60943,NaN,NaN,NaN,NaN,426368_con_02,διοικητικο,605,διοικητικα,0.997163,2577
55273,61038,NaN,NaN,NaN,NaN,936695_con_02,διοικητικο,605,διοικητικα,0.997163,2577
55274,61046,NaN,NaN,NaN,NaN,228930_con_03,διοικητικου,605,διοικητικου,1.000000,6954


In [46]:
final_entities['id'] = final_entities['id'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)

In [47]:
final_entities

,index,id,ihu_unique_step_id,ihu_unique_step_digital_id,ihu_unique_evidence_id,ihu_unique_condition_id,word,topic,Final_Entity,Similarity_Measure,Entity ID
0,4,963466,NaN,NaN,NaN,NaN,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ...",-1,"κεντρων ληψης, ελεγχου και διαβιβασης σηματων ...",1.000000,3839
1,18,268173,NaN,NaN,NaN,NaN,και επισκευαστες κλειδαριων,-1,και επισκευαστες κλειδαριων,1.000000,10036
2,63,871426,NaN,NaN,NaN,NaN,εθνικο μητρωο εθελοντων αιμοδοτων,-1,εθνικο μητρωο εθελοντων αιμοδοτων,1.000000,8239
3,72,233198,NaN,NaN,NaN,NaN,ειδικους εκλογικους καταλογους,-1,ειδικους εκλογικους καταλογους,1.000000,7880
4,80,251271,NaN,NaN,NaN,NaN,μητρωο αρνητων δωρεας,-1,μητρωο αρνητων δωρεας,1.000000,5342
...,...,...,...,...,...,...,...,...,...,...,...
55271,60867,NaN,NaN,NaN,NaN,948275_con_01,οργανωσης,605,οργανωσης,1.000000,569
55272,60943,NaN,NaN,NaN,NaN,426368_con_02,διοικητικο,605,διοικητικα,0.997163,2577
55273,61038,NaN,NaN,NaN,NaN,936695_con_02,διοικητικο,605,διοικητικα,0.997163,2577
55274,61046,NaN,NaN,NaN,NaN,228930_con_03,διοικητικου,605,διοικητικου,1.000000,6954


In [23]:
upload_ids_to_target_folder(entities_dict, "final_entities_dic")

split into dataframes and save

In [24]:
def filter_dataframe_by_id(df, column_name):
    # Filter the rows where "ihu_unique_step_id" is not "N/A"
    filtered_df = df[df[column_name].notnull()]

    # Select the desired columns for the new dataframe
    selected_columns = [column_name, "Final_Entity", "Entity ID"]
    new_df = filtered_df[selected_columns].reset_index(drop=True)

    return new_df

In [25]:
def upload_entity_files_to_target_folder(unique_id, name):

  result = filter_dataframe_by_id(final_entities, unique_id)

  path = name+".csv"

  result.to_csv(path, index=False)

  url=folder_analysis_url_shrpt+"entities_extraction"
  target_folder = ctx.web.get_folder_by_server_relative_url(url)
  with open(path, "rb") as content_file:
      file_content = content_file.read()
      target_folder.upload_file(os.path.basename(path), file_content).execute_query()

In [26]:
upload_entity_files_to_target_folder("ihu_unique_step_id", "steps_entities")

In [27]:
upload_entity_files_to_target_folder("ihu_unique_step_digital_id", "steps_digital_entities")

In [48]:
upload_entity_files_to_target_folder("id", "process_entities")

In [29]:
upload_entity_files_to_target_folder("ihu_unique_evidence_id", "evidences_entities")

In [30]:
upload_entity_files_to_target_folder("ihu_unique_condition_id", "conditions_entities")